# 4. Twitter - Sentiment Analysis

In [2]:
#install dependencies
!pip install textblob
!pip install tweepy
!pip install pycountry
!pip install wordcloud
!pip install langdetect
!pip install nltk
!pip install twython

In [96]:
#import libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [97]:
#read file
df_twitter_list = pd.read_csv('./API_twitter.csv') 
df_twitter_list.head()

,"Tesla, Inc.",SPDR S&P 500 ETF Trust,Apple Inc.,"Advanced Micro Devices, Inc.",Aphria Inc.,"AMC Entertainment Holdings, Inc.",NVIDIA Corporation,Alibaba Group Holding Limited,GameStop Corp.,Johnson & Johnson,"Cloudflare, Inc.",Roblox Corporation,Vanguard S&P 500 ETF,iShares Global Clean Energy ETF,Invesco QQQ Trust,Palantir Technologies Inc.
0,@montana_skeptic @NHSeacoast2 @elonmusk @Tesla...,SPDR S&amp;P 500 ETF Trust: Pivot points\n$SPY,@DJ_TUNEZ @DJSPINALL @fireboydml The bop with ...,"Advanced Micro Devices - Appeal, Reversed $AMD...","RT @doralgator: $APHA ""APHRIA INC. will be hos...",@DCashInvest @DCashInvest I just use and refre...,Huang is the co-founder – and president and CE...,Alibaba Group Holding Limited current quarter ...,RT @StonkstoMoon: Pretty sure that it’s all up...,"RT @Bristol_Phil: “Prime minister, I think you...",Cloudflare Inc $NET: Sales Growth Of 50%. Grow...,@Nate04909897 @harry5432g @DaRealAlterAffe @Kr...,@theficouple @Budgetdog_ @DefiningWealth Also ...,🌱🔋 $ICLN iShares Global Clean Energy ETF\nvs.\...,Invesco QQQ Trust: Pivot points\n$QQQ,"RT @LunarRoot: ""In the past four years, execut..."
1,BUYsignal Tesla Inc at 752.00 Signaltime was ...,SPY: SPDR® S&amp;P 500® ETF Trust https://t.co...,🎧 Ep28 of Jo Durrant’s Beautiful Universe podc...,$AMD #Advanced Micro Devices Inc. ... Next Gen...,"$APHA ""APHRIA INC. will be hosting a Virtual S...",AMC Entertainment Holdings Inc. Cl A stock ris...,Published a new blog entry Automotive Augmente...,Massive Growth in B2B Content Marketing Market...,"GameStop Corp, $GME, is up 11.6% - https://t.c...",RT @DbienaimeNYDN: Not Colin trying to say Jul...,DeItaone: CLOUDFLARE INC - PARTNERING WITH NVIDIA,@harry5432g @DaRealAlterAffe @KreekCraft Roblo...,RT @theficouple: Maybe you’re not going to be ...,🌱🔋 $GRID First Trust NASDAQ Clean Edge Smart G...,Invesco QQQ Trust: Pivot points\n$QQQ,"@MsScotsCrypto77 ""When will Coinbase go public..."
2,Green #vehicle producer #Tesla $TSLA claws its...,SPDR S&amp;P 500 ETF Trust: Pivot points\nThe ...,1991 Apple Inc Microphone NEW in Box 699-510...,Are You a Day Trader or a Swing Trader❓\n-\n🔊 ...,RT @aphriainc: Each Aphria Inc. #vape is desig...,RT @RSIDivergences: $AMC (AMC Entertainment Ho...,"RT @AKTU_Lucknow: CAS, @AKTU_Lucknow is organ...",RT @WitlinkA: 😊👉SenseTime has become the world...,RT @realMC_Capital: GameStop Corp said Tuesday...,"RT @marcuschown: Dear @BBCr4today, there is no...",RT @phishunt_io: #NewPhishing | #phishing #sca...,"Also, if you want to check it out, play this g...",Maybe you’re not going to be Tom Brady\n\nYou’...,🌱🔋 $ICLN iShares Global Clean Energy ETF\nvs.\...,Invesco QQQ Trust: Pivot points\n$QQQ,@CryptoGodJohn When will Coinbase go public? C...
3,RT @fossbytes14: Tesla is the most valuable pl...,SPDR S&amp;P 500 ETF Trust: Pivot points\n$SPY,Cryptocurrency ($2.18T) just matches Apple Inc...,Advanced Micro Devices Inc (AMD) price at clos...,RT @tilray: MAKE YOUR VOTE COUNT! $TLRY Remind...,$AMC (AMC Entertainment Holdings Inc) \n\nBull...,NVIDIA Corporation (NVDA) Investor Presentatio...,RT @WitlinkA: 😊👉SenseTime has become the world...,GameStop Corp said Tuesday it is redeeming sen...,RT @marcuschown: Retweet if you think speaker ...,#NewPhishing | #phishing #scam\n\n🌐 /sosogg.to...,@XtrlDex @SLabertat @Bloxy_News I doesnt make ...,The Vanguard S&amp;P 500 ETF (VOO) information...,🌱🔋 $QCLN First Trust NASDAQ Clean Edge Green E...,Invesco QQQ Trust: Pivot points\n$QQQ,@CryptoChihiro @eToro @eToro_DACH The Coinbase...
4,RT @binance: Introducing tradable stock tokens...,SPDR S&amp;P 500 ETF Trust: Pivot points\n$SPY,RT @BangtansVoting_: 🗳 STARPASS APP TUTORIAL\n...,Advanced Micro Devices Inc (AMD) stock rose by...,Aphria Inc. Announces Third Quarter Fiscal Yea...,B. Riley Financial upgraded AMC Entertainment ...,Harvesting AI: Startup's Weed Recognition for ...,RT @WitlinkA: 😊👉SenseTime has become the world...,📈 Today's Trending Stocks 📉\n\n$TSLA Tesla Inc...,@FOX19 Several states pause #Joh

In [98]:
#data cleaning

#fill in na with 'none'
df_twitter_list= df_twitter_list.fillna('none')

In [99]:
def percentage(part,whole):
    return 100 * float(part)/float(whole)

In [100]:
def sentiment(col):
    tweet_list = []
    neutral_list = []
    negative_list = []
    positive_list = []

    noOfTweet = 100
    positive = 0
    negative = 0
    neutral = 0
    polarity = 0

    for i in range(len(df_twitter_list[col])):
            #print(tweet.text)
        tweet_list.append(df_twitter_list[col][i])
        analysis = TextBlob(df_twitter_list[col][i])
        score = SentimentIntensityAnalyzer().polarity_scores(df_twitter_list[col][i])
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']
        polarity += analysis.sentiment.polarity
        if neg > pos:
            negative_list.append(df_twitter_list[col][i])
            negative += 1
        elif pos > neg:
            positive_list.append(df_twitter_list[col][i])
            positive += 1
        elif pos == neg:
            neutral_list.append(df_twitter_list[col][i])
            neutral += 1
    positive = format(percentage(positive, noOfTweet),'.1f')
    negative = format(percentage(negative, noOfTweet),'.1f')
    neutral = format(percentage(neutral, noOfTweet),'.1f')
    polarity = percentage(polarity, noOfTweet)
    tweet_list = pd.DataFrame(tweet_list)
    neutral_list = pd.DataFrame(neutral_list)
    negative_list = pd.DataFrame(negative_list)
    positive_list = pd.DataFrame(positive_list)
#     print("total number: ",len(tweet_list))
#     print("positive number: ",len(positive_list))
#     print("negative number: ", len(negative_list))
#     print("neutral number: ",len(neutral_list)) 
    tweet_list.drop_duplicates(inplace = True)

    #Cleaning Text (RT, Punctuation etc)
    #Creating new dataframe and new features
    tw_list = pd.DataFrame(tweet_list)
    tw_list["text"] = tw_list[0]
    #Removing RT, Punctuation etc
    remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
    rt = lambda x: re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", x)
    tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
    tw_list["text"] = tw_list.text.str.lower()
    #Calculating Negative, Positive, Neutral and Compound values
    tw_list[['polarity', 'subjectivity']] = tw_list[0].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
    for index, row in tw_list[0].iteritems():
        score = SentimentIntensityAnalyzer().polarity_scores(row)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']
        if neg > pos:        
            tw_list.loc[index, 'sentiment'] = 'negative'
        elif pos > neg:
            tw_list.loc[index, 'sentiment'] = 'positive'
        else:
            tw_list.loc[index, 'sentiment'] = 'neutral'
            tw_list.loc[index, 'neg'] = neg
            tw_list.loc[index, 'neu'] = neu
            tw_list.loc[index, 'pos'] = pos
            tw_list.loc[index, 'compound'] = comp
    return tw_list['sentiment']

In [101]:
columns = df_twitter_list.columns.to_list()
new_col = [i.split('(')[0].strip()+'_sentiment' for i in columns]

In [102]:
for i in range(len(new_col)):
    df_twitter_list[new_col[i]] = sentiment(columns[i])

In [103]:
df_twitter_list.head()

,"Tesla, Inc.",SPDR S&P 500 ETF Trust,Apple Inc.,"Advanced Micro Devices, Inc.",Aphria Inc.,"AMC Entertainment Holdings, Inc.",NVIDIA Corporation,Alibaba Group Holding Limited,GameStop Corp.,Johnson & Johnson,"Cloudflare, Inc.",Roblox Corporation,Vanguard S&P 500 ETF,iShares Global Clean Energy ETF,Invesco QQQ Trust,Palantir Technologies Inc.,"Tesla, Inc._sentiment",SPDR S&P 500 ETF Trust_sentiment,Apple Inc._sentiment,"Advanced Micro Devices, Inc._sentiment",Aphria Inc._sentiment,"AMC Entertainment Holdings, Inc._sentiment",NVIDIA Corporation_sentiment,Alibaba Group Holding Limited_sentiment,GameStop Corp._sentiment,Johnson & Johnson_sentiment,"Cloudflare, Inc._sentiment",Roblox Corporation_sentiment,Vanguard S&P 500 ETF_sentiment,iShares Global Clean Energy ETF_sentiment,Invesco QQQ Trust_sentiment,Palantir Technologies Inc._sentiment
0,@montana_skeptic @NHSeacoast2 @elonmusk @Tesla...,SPDR S&amp;P 500 ETF Trust: Pivot points\n$SPY,@DJ_TUNEZ @DJSPINALL @fireboydml The bop with ...,"Advanced Micro Devices - Appeal, Reversed $AMD...","RT @doralgator: $APHA ""APHRIA INC. will be hos...",@DCashInvest @DCashInvest I just use and refre...,Huang is the co-founder – and president and CE...,Alibaba Group Holding Limited current quarter ...,RT @StonkstoMoon: Pretty sure that it’s all up...,"RT @Bristol_Phil: “Prime minister, I think you...",Cloudflare Inc $NET: Sales Growth Of 50%. Grow...,@Nate04909897 @harry5432g @DaRealAlterAffe @Kr...,@theficouple @Budgetdog_ @DefiningWealth Also ...,🌱🔋 $ICLN iShares Global Clean Energy ETF\nvs.\...,Invesco QQQ Trust: Pivot points\n$QQQ,"RT @LunarRoot: ""In the past four years, execut...",negative,positive,neutral,positive,neutral,neutral,neutral,positive,positive,neutral,positive,negative,positive,positive,positive,neutral
1,BUYsignal Tesla Inc at 752.00 Signaltime was ...,SPY: SPDR® S&amp;P 500® ETF Trust https://t.co...,🎧 Ep28 of Jo Durrant’s Beautiful Universe podc...,$AMD #Advanced Micro Devices Inc. ... Next Gen...,"$APHA ""APHRIA INC. will be hosting a Virtual S...",AMC Entertainment Holdings Inc. Cl A stock ris...,Published a new blog entry Automotive Augmente...,Massive Growth in B2B Content Marketing Market...,"GameStop Corp, $GME, is up 11.6% - https://t.c...",RT @DbienaimeNYDN: Not Colin trying to say Jul...,DeItaone: CLOUDFLARE INC - PARTNERING WITH NVIDIA,@harry5432g @DaRealAlterAffe @KreekCraft Roblo...,RT @theficouple: Maybe you’re not going to be ...,🌱🔋 $GRID First Trust NASDAQ Clean Edge Smart G...,Invesco QQQ Trust: Pivot points\n$QQQ,"@MsScotsCrypto77 ""When will Coinbase go public...",neutral,positive,positive,positive,neutral,positive,neutral,positive,neutral,positive,neutral,positive,neutral,positive,NaN,negative
2,Green #vehicle producer #Tesla $TSLA claws its...,SPDR S&amp;P 500 ETF Trust: Pivot points\nThe ...,1991 Apple Inc Microphone NEW in Box 699-510...,Are You a Day Trader or a Swing Trader❓\n-\n🔊 ...,RT @aphriainc: Each Aphria Inc. #vape is desig...,RT @RSIDivergences: $AMC (AMC Entertainment Ho...,"RT @AKTU_Lucknow: CAS, @AKTU_Lucknow is organ...",RT @WitlinkA: 😊👉SenseTime has become the world...,RT @realMC_Capital: GameStop Corp said Tuesday...,"RT @marcuschown: Dear @BBCr4today, there is no...",RT @phishunt_io: #NewPhishing | #phishing #sca...,"Also, if you want to check it out, play this g...",Maybe you’re not going to be Tom Brady\n\nYou’...,🌱🔋 $ICLN iShares Global Clean Energy ETF\nvs.\...,Invesco QQQ Trust: Pivot points\n$QQQ,@CryptoGodJohn When will Coinbase go public? C...,positive,positive,neutral,negative,neutral,positive,neutral,positive,positive,negative,neutral,positive,positive,positive,NaN,negative
3,RT @fossbytes14: Tesla is the most valuable pl...,SPDR S&amp;P 500 ETF Trust: Pivot points\n$SPY,Cryptocurrency ($2.18T) just matches Apple Inc...,Advanced Micro Devices Inc (AMD) price at clos...,RT @tilray: MAKE YOUR VOTE COUNT! $TLRY Remind...,$AMC (AMC Entertainment Holdings Inc) \n\nBull...,NVIDIA Corporation (NVDA) Investor Presentatio...,RT @Witli

In [1]:
#save dataframe into csv
df_twitter_list.to_csv('./twitter_sentiment_analysis.csv', sep=',', header=True, index=False)

In [24]:
#further sentiment analysis 

#change file path here
df_twitter_list = pd.read_csv('./twitter_sentiment_analysis.csv') 
df_twitter_list.head()

#create new dataframe with only sentiments
df_twitter_sentiment = df_twitter_list.iloc[: , 16:]
df_twitter_sentiment.head()

,"Tesla, Inc._sentiment",SPDR S&P 500 ETF Trust_sentiment,Apple Inc._sentiment,"Advanced Micro Devices, Inc._sentiment",Aphria Inc._sentiment,"AMC Entertainment Holdings, Inc._sentiment",NVIDIA Corporation_sentiment,Alibaba Group Holding Limited_sentiment,GameStop Corp._sentiment,Johnson & Johnson_sentiment,"Cloudflare, Inc._sentiment",Roblox Corporation_sentiment,Vanguard S&P 500 ETF_sentiment,iShares Global Clean Energy ETF_sentiment,Invesco QQQ Trust_sentiment,Palantir Technologies Inc._sentiment
0,negative,positive,neutral,positive,neutral,neutral,neutral,positive,positive,neutral,positive,negative,positive,positive,positive,neutral
1,neutral,positive,positive,positive,neutral,positive,neutral,positive,neutral,positive,neutral,positive,neutral,positive,NaN,negative
2,positive,positive,neutral,negative,neutral,positive,neutral,positive,positive,negative,neutral,positive,positive,positive,NaN,negative
3,positive,NaN,neutral,positive,neutral,positive,neutral,NaN,positive,negative,neutral,neutral,negative,positive,NaN,positive
4,positive,NaN,neutral,positive,neutral,positive,neutral,NaN,neutral,neutral,neutral,neutral,positive,positive,positive,negative


In [25]:
#transform sentiments into numeric form for easy manipulation 
#where positive is 1, neutral is zero and negative is -1
df_twitter_sentiment = df_twitter_sentiment.replace({'positive':1, 'neutral':0, 'negative':-1})
df_twitter_sentiment= df_twitter_sentiment.fillna(0)

df_twitter_sentiment.head(10)

,"Tesla, Inc._sentiment",SPDR S&P 500 ETF Trust_sentiment,Apple Inc._sentiment,"Advanced Micro Devices, Inc._sentiment",Aphria Inc._sentiment,"AMC Entertainment Holdings, Inc._sentiment",NVIDIA Corporation_sentiment,Alibaba Group Holding Limited_sentiment,GameStop Corp._sentiment,Johnson & Johnson_sentiment,"Cloudflare, Inc._sentiment",Roblox Corporation_sentiment,Vanguard S&P 500 ETF_sentiment,iShares Global Clean Energy ETF_sentiment,Invesco QQQ Trust_sentiment,Palantir Technologies Inc._sentiment
0,-1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,-1.0,1.0,1.0,1.0,0.0
1,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.0
2,1.0,1.0,0.0,-1.0,0.0,1.0,0.0,1.0,1.0,-1.0,0.0,1.0,1.0,1.0,0.0,-1.0
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,0.0,0.0,-1.0,1.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,-1.0
5,0.0,1.0,1.0,1.0,1.0,-1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
6,0.0,-1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,-1.0,-1.0,1.0,0.0,1.0
7,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
8,-1.0,1.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
9,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.0,0.0,1.0,-1.0,1.0,1.0,1.0


In [26]:
#get average sentiments for each stock
df_twitter_average_sentiment = df_twitter_sentiment.mean() 
print(df_twitter_average_sentiment)

Tesla, Inc._sentiment                         0.17
SPDR S&P 500 ETF Trust_sentiment              0.06
Apple Inc._sentiment                          0.25
Advanced Micro Devices, Inc._sentiment        0.34
Aphria Inc._sentiment                         0.12
AMC Entertainment Holdings, Inc._sentiment    0.27
NVIDIA Corporation_sentiment                  0.21
Alibaba Group Holding Limited_sentiment       0.01
GameStop Corp._sentiment                      0.20
Johnson & Johnson_sentiment                  -0.04
Cloudflare, Inc._sentiment                    0.07
Roblox Corporation_sentiment                  0.11
Vanguard S&P 500 ETF_sentiment                0.01
iShares Global Clean Energy ETF_sentiment     0.58
Invesco QQQ Trust_sentiment                   0.06
Palantir Technologies Inc._sentiment          0.15
dtype: float64


In [27]:
#save average sentiments results into csv
df_twitter_average_sentiment.to_csv('./twitter_average_sentiment.csv', sep=',', index=True)